In [8]:
import googlemaps
from datetime import datetime
from bs4 import BeautifulSoup




def get_key(api_key_path):
    """提供key檔案路徑，自動讀取內容並回傳"""
    with open(api_key_path, "r", encoding="UTF-8") as f:
        api_key = f.read()
    
    return api_key


def trans_unix_to_date(timestamp):
    """將UNIX電腦時間轉換為西元年月日"""
    # 將時間戳轉成 datetime 物件
    dt = datetime.fromtimestamp(timestamp)

    # 格式化成 "年-月-日"
    date_str = dt.strftime('%Y-%m-%d')

    return date_str

# def if_exist(value):
#     if value:
#         return value
#     else:
#         return None


def get_place_id(api_key: str, key_word: str):
    """提供關鍵字（最好是店名+地址），透過gmap搜尋取得place_id"""
    gmaps = googlemaps.Client(key=api_key)
    search_result = gmaps.places(query=key_word, language='zh-TW')
    if len(search_result["results"]) != 0:
        place_id = search_result.get("results", {})[0]["place_id"]
        return place_id
    else:
        return None



def newest_review_date(review_list: list):
    """提供gmaps回傳的評論列表，回傳列表中最新的評論時間"""
    for i in review_list:
        time = 0
        if i["time"] > 0:
            time = i["time"]
    date = trans_unix_to_date(time)

    return date


def gmap_info(ori_name, api_key, place_id):
    """提供place_id，回傳名稱、營業狀態、營業時間、gmap評分、經緯度、gmap網址、最新評論日期"""
    if place_id != None:
        gmaps = googlemaps.Client(key=api_key)
        detail = gmaps.place(place_id=place_id, language='zh-TW')
        name = detail.get("result", {}).get("name", None)
        business_status = detail.get("result", {}).get("business_status", None)

        if detail["result"]["formatted_address"]:
            address = detail.get("result", {}).get("formatted_address", None)
        elif detail['result']['adr_address']:
            address = BeautifulSoup(detail.get('result', {}).get('adr_address', None), "html.parser").text
        else:
            address = None
        
        phone = detail.get("result", {}).get("formatted_phone_number", None)
        if phone != None:
            phone = phone.replace(" ", "")

        opening_hours = detail.get("result", {}).get("opening_hours", {}).get("weekday_text", None)
        rating = detail.get("result", {}).get('rating', None)
        rating_total = detail.get("result", {}).get("user_ratings_total", None)
        # geocode = if_exist(detail["result"]['geometry']['location'])
        longitude = detail.get("result", {}).get('geometry', {}).get('location', {}).get('lng', None)
        latitude = detail.get("result", {}).get('geometry', {}).get('location', {}).get('lat', None)
        map_url = detail.get("result", {}).get("url", None)
        review_list = detail.get("result", {}).get('reviews', None)
        
        if review_list != None:
            newest_review = newest_review_date(review_list)
        else:
            newest_review = None

        place_info = {
            "name":name,
            "place_id":place_id,
            "business_status":business_status,
            "address":address,
            "phone":phone,
            "opening_hours":opening_hours,
            "rating":rating,
            "rating_total":rating_total,
            "longitude":longitude,
            "latitude":latitude,
            "map_url":map_url,
            "newest_review":newest_review
        }
    else:
        place_info = {
            "name":ori_name,
            "place_id":None,
            "business_status":None,
            "address":None,
            "phone":None,
            "opening_hours":None,
            "rating":None,
            "rating_total":None,
            "longitude":None,
            "latitude":None,
            "map_url":None,
            "newest_review":None
        }

    return place_info


def get_place_dict(api_key_path, name: str, address: str):
    key_word = name + " " + address

    api_key = get_key(api_key_path=api_key_path)
    place_id = get_place_id(api_key=api_key, key_word=key_word)
    place_dict = gmap_info(ori_name=name, api_key=api_key, place_id=place_id)

    return place_dict

In [9]:
import pandas as pd

data_path = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\petboo_fnb_retest.csv"
df = pd.read_csv(data_path)

df_sample = df.iloc[10:16]

df_sample

,city,district,name,address,phone,rating,rating_count,pet_rules,url
10,台北市,中正區,貳樓餐廳 微風台北車站店,100台灣台北市中正區北平西路3號2F,02 2361 0861,4.2,2431.0,可落地、寵物友善,https://taiwan.petboo.co/place/1313
11,台北市,中正區,Little M cafe,100台灣台北市中正區和平西路一段57號1樓,02 2358 1821,4.7,113.0,可落地、寵物友善,https://taiwan.petboo.co/place/1182
12,台北市,中正區,魚刺人雞蛋糕台北師大店,100台灣台北市中正區羅斯福路二段30巷13號,02 2322 1718,4.3,94.0,可落地、寵物友善,https://taiwan.petboo.co/place/1075
13,台北市,中正區,新馬辣經典麻辣鍋-公館店,100台灣台北市中正區汀州路三段295號,02 2364 6215,4.5,5238.0,可落地、寵物友善,https://taiwan.petboo.co/place/868
14,台北市,中正區,Mr. 雪腐 公館店,100台灣台北市中正區羅斯福路三段244巷21號,02 2363 5200,4.5,1841.0,可落地、寵物友善,https://taiwan.petboo.co/place/863
15,台北市,中正區,沐樂咖啡（寵物友善咖啡廳）,100台灣台北市中正區臨沂街13巷7號,02 2358 7288,4.0,915.0,可落地、寵物友善,https://taiwan.petboo.co/place/735


In [10]:
api_key_path = r"C:\Users\add41\Documents\Data_Engineer\Project\TJR103-Group-1-Project\code\1_bevis\tool\map_api_key.txt"

data_list = []
for index, row in df_sample.iterrows():
    name = row["name"]
    address = row["address"]
    data_list.append(get_place_dict(api_key_path, name, address))

In [11]:
df_final = pd.DataFrame(data_list)
df_final

,name,place_id,business_status,address,phone,opening_hours,rating,rating_total,longitude,latitude,map_url,newest_review
0,Second Floor 貳樓微風台北車站店,ChIJneUTn3KpQjQR8_eBLgiNFMU,OPERATIONAL,100台灣台北市中正區北平西路3號2樓,0223610861,"[星期一: 10:00 – 22:00, 星期二: 10:00 – 22:00, 星期三: ...",4.3,3979,121.517044,25.047749,https://maps.google.com/?cid=14201130591285540851,2025-05-30
1,Little M cafe,ChIJoyIVxD2pQjQR7G2vfTCGVTM,OPERATIONAL,100台灣台北市中正區和平西路一段57號1樓,0223581821,"[星期一: 10:00 – 19:00, 星期二: 10:00 – 19:00, 星期三: ...",4.8,398,121.519762,25.026704,https://maps.google.com/?cid=3699010211779931628,2025-03-29
2,魚刺人雞蛋糕台北師大店,ChIJ_2j3P_-pQjQR6Cs6xWb4TDg,CLOSED_PERMANENTLY,100台灣台北市中正區羅斯福路二段30巷13號,0223221718,None,4.3,325,121.520884,25.027999,https://maps.google.com/?cid=4056890484633250792,2022-05-12
3,新馬辣經典麻辣鍋-公館店,ChIJp4DmtIqpQjQRMc1gEd_aAS4,OPERATIONAL,100台灣台北市中正區汀州路三段295號,0223646215,"[星期一: 11:30 – 00:00, 星期二: 11:30 – 00:00, 星期三: ...",4.6,9464,121.535785,25.011975,https://maps.google.com/?cid=3315171452325514545,2025-08-29
4,Mr. 雪腐 公館店,ChIJm6NFboypQjQR-FXOgyKvjA4,OPERATIONAL,100台灣台北市中正區羅斯福路三段244巷21號,0223635200,"[星期一: 12:15 – 22:00, 星期二: 12:15 – 22:00, 星期三: ...",4.6,3213,121.530457,25.016654,https://maps.google.com/?cid=1048405376045569528,2025-09-11
5,沐樂咖啡,ChIJYZL7fnupQjQRmYAxg8hOXuo,OPERATIONAL,100台灣台北市中正區臨沂街13巷7號,0223587288,"[星期一: 11:00 – 18:30, 星期二: 11:00 – 18:30, 星期三: ...",4.1,1239,121.531086,25.042352,https://maps.google.com/?cid=16888022275787423897,2025-04-10
